In [4]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Patch
import subprocess
import sys

In [ ]:
use_hatches = True

# EER values for each system, augmentation, and set (dev/test)
# Format: [dev, test]
data = {
    "T10-2": {"No Aug.": [31.00, 28.27], "B3": [26.58, 23.45], "B4": [23.50, 19.19],
              "B5": [26.01, 23.02], "SpecAugment": [26.58, 25.14], "A.20": [23.75, 22.55]},
    "T12-5": {"No Aug.": [29.25, 25.49], "B3": [28.00, 24.37], "B4": [28.06, 23.94],
              "B5": [27.84, 24.58], "SpecAugment": [27.87, 23.83], "A.20": [28.75, 25.50]},
    "T25-1": {"No Aug.": [30.43, 27.66], "B3": [30.15, 27.26], "B4": [29.55, 27.45],
              "B5": [29.03, 26.97], "SpecAugment": [29.25, 27.12], "A.20": [29.05, 27.70]}
}

# Set up publication-quality plot styling
rcParams = {
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'font.size': 26,
    'axes.labelsize': 26,
    'axes.titlesize': 28,
    'xtick.labelsize': 24,
    'ytick.labelsize': 24,
    'legend.fontsize': 24,
    'figure.figsize': (21, 12),
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.05
}

In [ ]:
def is_latex_available():
    try:
        # Try to compile a minimal LaTeX document
        test_latex = subprocess.run(
            ['latex', '-interaction=nonstopmode', '-halt-on-error'],
            input=r'''\documentclass{minimal}
                      \begin{document}
                      Test
                      \end{document}'''.encode(),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=3
        )
        return test_latex.returncode == 0
    except (subprocess.SubprocessError, FileNotFoundError):
        return False

# Determine whether to use LaTeX for text rendering
use_latex = True
try:
    # Try importing type1cm
    import importlib
    importlib.util.find_spec('type1cm')
except ImportError:
    print("Warning: LaTeX dependencies not found. Falling back to standard text rendering.")
    use_latex = False


if use_latex and is_latex_available():
    print("Using LaTeX rendering for text")
    rcParams.update({
        'text.usetex': True,
        'text.latex.preamble': r'\usepackage{amsmath} \usepackage{amssymb}'
    })
else:
    print("Using standard Matplotlib text rendering")
    rcParams.update({
        'text.usetex': True
    })
    use_latex = True

plt.rcParams.update(rcParams)

# Data from the table
systems = ["T10-2", "T12-5", "T25-1"]
augmentations = ["B3", "B4", "B5", "SpecAugment"]
baselines = ["A.20", "No Aug."]


# Compute best performers
best_dev = {sys: min(data[sys], key=lambda m: data[sys][m][0]) for sys in systems}
best_test = {sys: min(data[sys], key=lambda m: data[sys][m][1]) for sys in systems}


# Create two subplots for development and test sets
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(21, 12))

# Set up width of bars and positions
bar_width = 0.14  # Made narrower to accommodate more bars
index = np.arange(len(systems))

# Create colorblind-friendly color palette - extended for the new methods
# colors = ['#377eb8', '#ff7f00', '#4daf4a', '#984ea3', '#e41a1c', '#a65628']
colors = ['#5b8db8',  # Soft Blue
          '#f4a261',  # Warm Apricot
          '#81b29a',  # Muted Green
          '#b07aa1',  # Soft Purple
          '#e76f51',  # Mellow Red
          '#a0714f']  # Earthy Brown


# Combine all methods for plotting
all_methods = augmentations + baselines

# Plot data for development set
for i, aug in enumerate(all_methods):
    dev_values = [data[system][aug][0] for system in systems]
    bars = ax1.bar(index + i * bar_width - 1.5 * bar_width, dev_values, bar_width,
            label=aug, color=colors[i], edgecolor='black', linewidth=0.8)
    
    # Add stars to bars representing best performance for each system
    for j, system in enumerate(systems):
        if aug == best_dev[system]:
            ax1.scatter(j + i * bar_width - 1.5 * bar_width, dev_values[j] - 0.8, 
                       marker='*', s=150, color='black', zorder=10)

# Plot data for test set
for i, aug in enumerate(all_methods):
    test_values = [data[system][aug][1] for system in systems]
    bars = ax2.bar(index + i * bar_width - 1.5 * bar_width, test_values, bar_width,
            label=aug, color=colors[i], edgecolor='black', linewidth=0.8)
    
    # Add stars to bars representing best performance for each system
    for j, system in enumerate(systems):
        if aug == best_test[system]:
            ax2.scatter(j + i * bar_width - 1.5 * bar_width, test_values[j] - 0.8, 
                       marker='*', s=150, color='black', zorder=10)

# Customize the plots
counter = 0

# Customize the plots
for ax, title in zip([ax1, ax2], ["Dev. set", "Test set"]):
    if use_latex:
        y_label = r'$\textrm{EER}$ [$\%$] $(\downarrow)$'
    else:
        y_label = 'EER[%] (↓)'

    if ax == ax1:
        ax.set_ylabel(y_label)

    ax.set_title(title)
    ax.set_xticks(index)
    ax.set_xticklabels(systems)
    ax.set_ylim(10, 32)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Add value labels and hatching
    bars = ax.patches
    patterns = ['/', '\\', '+', 'x', '-', '--']
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2, height + 0.3,
            f'{height:.1f}', ha='center', va='bottom',
            fontsize=16, rotation=0, fontweight='bold'
        )
        if use_hatches:
            bar.set_hatch(patterns[i // len(systems)])

    # Add vertical separators between SpecAugment and A.20 in each group
    for system_idx in index:
        separator_x = system_idx + 3.5 * bar_width - 1.5 * bar_width
        ax.axvline(x=separator_x, color='gray', linestyle='--', linewidth=1.2, alpha=0.7)

        # Optional label for one of the separators (e.g., only for the first system)
        if system_idx == 0:
            ax.text(separator_x + 0.02, ax.get_ylim()[1] - 1.5, 'Benchmarks',
                    rotation=90, fontsize=18, va='top', ha='left', color='gray')



# Create custom legend with both colors and patterns
legend_elements = []
for i, method in enumerate(all_methods):
    if use_hatches:
        legend_elements.append(Patch(facecolor=colors[i], label=method, 
                                     edgecolor='black', hatch=patterns[i]))
    else:
        legend_elements.append(Patch(facecolor=colors[i], label=method, 
                                     edgecolor='black'))

# Add explanatory note about star marker
if use_latex:
    star_label = r'$\star$ Best'
else:
    star_label = '* Best performance'
legend_elements.append(Patch(facecolor='white', edgecolor='white', label=star_label))

# Add a single legend for both subplots
fig.legend(handles=legend_elements, loc='upper center', 
           bbox_to_anchor=(0.5, 0.04), ncol=4, frameon=True)

# Adjust layout and save the figure
plt.tight_layout()
plt.subplots_adjust(bottom=0.065)

# Try to save PDF, but handle errors gracefully
try:
    plt.savefig('system_performance_comparison.pdf')
    print("PDF saved successfully")
except Exception as e:
    print(f"Error saving PDF: {e}")
    print("Saving as PNG only")

# Save as PNG (more universally compatible)
plt.savefig('system_performance_comparison.png', dpi=300)
print("PNG saved successfully")

# Show the figure
plt.show()

In [ ]:
plt.rcParams.update(rcParams)

# Flag to toggle hatching
use_hatches = True

# Group identifiers and ordering
systems = list(data.keys())
augmentations = ["B3", "B4", "B5", "SpecAugment"]
baselines    = ["A.20", "No Aug."]
methods = augmentations + baselines

# Colors and hatches matching methods order
colors = ['#f4a261', '#81b29a', '#b07aa1', '#e76f51', '#a0714f', '#5b8db8']
hatches = ['/', '\\', '+', 'x', '-', '--']
bar_width = 0.14
index = np.arange(len(systems))

# Compute averages and best methods
avg_data = {sys: {m: np.mean(data[sys][m]) for m in methods} for sys in systems}
best_method = {sys: min(avg_data[sys], key=lambda m: avg_data[sys][m]) for sys in systems}

# Create figure
fig, ax = plt.subplots()

# Plot bars with optional hatches
for i, method in enumerate(methods):
    values = [avg_data[sys][method] for sys in systems]
    bars = ax.bar(
        index + (i - (len(methods)-1)/2) * bar_width,
        values,
        bar_width,
        label=method,
        color=colors[i],
        # edgecolor='black',
        edgecolor='#555555',
        linewidth=0.8,
        hatch=hatches[i] if use_hatches else None
    )
    # Mark best with gold star
    for j, sys in enumerate(systems):
        if method == best_method[sys]:
            x = index[j] + (i - (len(methods)-1)/2) * bar_width
            y = values[j]
            ax.scatter(x, y - 0.5, marker='*', s=200, color='black', linewidths=1, zorder=10)

# Separator before baselines
sep_offset = (len(augmentations) - 0.5 - (len(methods)-1)/2) * bar_width
for j in range(len(systems)):
    x = index[j] + sep_offset
    ax.axvline(x=x, color='gray', linestyle='--', linewidth=1.2, alpha=0.7)
    if j == 0:
        ax.text(
            x + 0.02, ax.get_ylim()[1] - 1.5, 'Benchmarks',
            rotation=90, fontsize=20, va='top', ha='left', color='#555555'
        )

# Labels and aesthetics
ax.set_xticks(index)
ax.set_xticklabels(systems)
ax.set_ylabel(r'$\textrm{EER}_{\textrm{avg}}$ [$\%$] $(\downarrow)$')
ax.set_ylim(15, 31.5)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.25)

# Annotate
for bar in ax.patches:
    h = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        h + 0.2,
        f'{h:.1f}', ha='center', va='bottom', fontsize=20, fontweight='bold'
    )

# Legend including star
legend_elems = [
    Patch(facecolor=colors[i], edgecolor='black', hatch=hatches[i] if use_hatches else '', label=methods[i])
    for i in range(len(methods))
]
legend_elems.append(Patch(facecolor='white', edgecolor='white', label=r'$\star$ Best'))

ax.legend(
    handles=legend_elems,
    loc='upper center',
    bbox_to_anchor=(0.5, -0.04),
    ncol=4,
    frameon=True
)

# Try to save PDF, but handle errors gracefully
try:
    plt.savefig('system_performance_comparison.pdf')
    print("PDF saved successfully")
except Exception as e:
    print(f"Error saving PDF: {e}")
    print("Saving as PNG only")


plt.tight_layout()
plt.savefig('system_performance_comparison.png', dpi=300)
plt.show()